# Companies - Taxonomy

Get the full list of companies included in FSS

In [71]:
import os
import requests as req
import pandas as pd
import fastavro
from io import BytesIO
from dotenv import load_dotenv
load_dotenv()

USER_KEY = os.getenv("FSS_USERKEY")
BASE_URL = "https://api.dowjones.com"

REQ_HEADERS = {
    "user-key": USER_KEY,
    "Accept": "application/json"
}

## CSV

In [38]:
csv_companies_url = f"{BASE_URL}/fss/taxonomies/companies?format=csv"

response = req.get(csv_companies_url, headers=REQ_HEADERS)
taxonomy_response = response.json()
csv_file_url = taxonomy_response["download_urls"][0]
csv_df = pd.read_csv(csv_file_url, storage_options=REQ_HEADERS)

/tmp/ipykernel_325913/3270192162.py:6: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_df = pd.read_csv(csv_file_url, storage_options=REQ_HEADERS)


In [39]:
csv_df.head()

,_CompanyCode,CompanyName,Address1,Address2,Address3,City,State,PostalCode,Country,Region,...,IndustryDescriptor,ListStatus,ExchangeTicker,Status,RegistrationID,URL,SEDOL,DUNS,CUSIP,ISIN
0,AABAG,Aachener Bausparkasse Aktiengesellschaft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GFR,...,Mortgage Banks/Real Estate Credit,UL,NaN,A,HRB88,http://www.aachener-bausparkasse.de,NaN,315719336,NaN,
1,ABLXIS,Ablexis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,...,Biotechnology Services,UL,NaN,A,NaN,NaN,NaN,051068549,NaN,
2,ADHARS,Adharshila Capital Services Ltd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INDIA,...,Financial Investment Services,UL,NaN,A,NaN,NaN,NaN,916517043,NaN,
3,AGBIOM,AgBiome LLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,...,Biotechnology Services,UL,NaN,A,NaN,NaN,NaN,079150527,NaN,
4,ALACRV,Aar,NaN,NaN,NaN,Moscow,NaN,NaN,NaN,RUSS,...,Financial Investment Services,UL,NaN,A,NaN,NaN,NaN,NaN,NaN,


## JSON

Important: The JSON format returned in the file contains a record per line. This requires to use the option `lines=True` in `pandas.read_json(...)`.

In [36]:
json_companies_url = f"{BASE_URL}/fss/taxonomies/companies?format=json"

response = req.get(json_companies_url, headers=REQ_HEADERS)
taxonomy_response = response.json()
json_file_url = taxonomy_response["download_urls"][0]
json_df = pd.read_json(json_file_url, orient='records', lines=True, storage_options=REQ_HEADERS)

In [37]:
json_df.head()

,_CompanyCode,CompanyName,Region,RegionDescriptor,Industry,IndustryDescriptor,ListStatus,Status,RegistrationID,URL,...,City,ExchangeTicker,SEDOL,CUSIP,Address1,PostalCode,Address2,State,Address3,Country
0,AABAG,Aachener Bausparkasse Aktiengesellschaft,GFR,Germany,I8150103,Mortgage Banks/Real Estate Credit,UL,A,HRB88,http://www.aachener-bausparkasse.de,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABLXIS,Ablexis,USA,United States,I2569,Biotechnology Services,UL,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ADHARS,Adharshila Capital Services Ltd,INDIA,India,I831,Financial Investment Services,UL,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AGBIOM,AgBiome LLC,USA,United States,I2569,Biotechnology Services,UL,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ALACRV,Aar,RUSS,Russia,I831,Financial Investment Services,UL,A,NaN,NaN,...,Moscow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## AVRO

In [72]:
def read_avro_from_url(url):
    response = req.get(url, headers=REQ_HEADERS)
    records = []

    avro_file = BytesIO(response.content)
    reader = fastavro.reader(avro_file)
    records = [r for r in reader]

    df = pd.DataFrame.from_records(records)

    df['URL'] = df['URL'].fillna('')
    df['ExchangeTicker'] = df['ExchangeTicker'].fillna('')
    df['ISIN'] = df['ISIN'].fillna('')
    return df


In [73]:
avro_companies_url = f"{BASE_URL}/fss/taxonomies/companies?format=avro"
response = req.get(avro_companies_url, headers=REQ_HEADERS)
taxonomy_response = response.json()
avro_file_url = taxonomy_response["download_urls"][0]
avro_df = read_avro_from_url(avro_file_url)

In [74]:
avro_df.shape

(148522, 22)

In [61]:
avro_df.head()

,_CompanyCode,CompanyName,Address1,Address2,Address3,City,State,PostalCode,Country,Region,...,IndustryDescriptor,ListStatus,ExchangeTicker,Status,RegistrationID,URL,SEDOL,DUNS,CUSIP,ISIN
0,AABAG,Aachener Bausparkasse Aktiengesellschaft,None,None,None,None,None,None,None,GFR,...,Mortgage Banks/Real Estate Credit,UL,,A,HRB88,http://www.aachener-bausparkasse.de,None,315719336,None,
1,ABLXIS,Ablexis,None,None,None,None,None,None,None,USA,...,Biotechnology Services,UL,,A,None,,None,051068549,None,
2,ADHARS,Adharshila Capital Services Ltd,None,None,None,None,None,None,None,INDIA,...,Financial Investment Services,UL,,A,None,,None,916517043,None,
3,AGBIOM,AgBiome LLC,None,None,None,None,None,None,None,USA,...,Biotechnology Services,UL,,A,None,,None,079150527,None,
4,ALACRV,Aar,None,None,None,Moscow,None,None,None,RUSS,...,Financial Investment Services,UL,,A,None,,None,None,None,


## Mapping companies

Operations that help mapping companies from the taxonomy to a custom list

In [75]:
print("Columns:")
for col in avro_df.columns:
    print(f"* {col}")

Columns:
* _CompanyCode
* CompanyName
* Address1
* Address2
* Address3
* City
* State
* PostalCode
* Country
* Region
* RegionDescriptor
* Industry
* IndustryDescriptor
* ListStatus
* ExchangeTicker
* Status
* RegistrationID
* URL
* SEDOL
* DUNS
* CUSIP
* ISIN


In [76]:
avro_df[avro_df.CompanyName.str.contains('apple, inc', case=False)]

,_CompanyCode,CompanyName,Address1,Address2,Address3,City,State,PostalCode,Country,Region,...,IndustryDescriptor,ListStatus,ExchangeTicker,Status,RegistrationID,URL,SEDOL,DUNS,CUSIP,ISIN
132885,APPLC,"Apple, Inc.",One Apple Park Way,None,None,Cupertino,California,95014-2083,United States,USA,...,Computers/Consumer Electronics,L,AAPL,A,None,http://www.apple.com,2046251,060704780,037833100,US0378331005
133542,GLLRSV,"Pineapple, Inc.",12301 Wilshire Boulevard,Suite 420,None,Los Angeles,California,90025,United States,USA,...,Marijuana Retailing,L,PNPL,A,None,http://www.pineappleexpress.com,BPLJQP7,062304913,72303J101,US72303J1016


In [67]:
avro_df[avro_df.ExchangeTicker == 'AAPL']

,_CompanyCode,CompanyName,Address1,Address2,Address3,City,State,PostalCode,Country,Region,...,IndustryDescriptor,ListStatus,ExchangeTicker,Status,RegistrationID,URL,SEDOL,DUNS,CUSIP,ISIN
132854,APPLC,"Apple, Inc.",One Apple Park Way,None,None,Cupertino,California,95014-2083,United States,USA,...,Computers/Consumer Electronics,L,AAPL,A,None,http://www.apple.com,2046251,060704780,037833100,US0378331005


In [68]:
avro_df[avro_df.ISIN.str.contains('US0378331005', case=False)]

,_CompanyCode,CompanyName,Address1,Address2,Address3,City,State,PostalCode,Country,Region,...,IndustryDescriptor,ListStatus,ExchangeTicker,Status,RegistrationID,URL,SEDOL,DUNS,CUSIP,ISIN
132854,APPLC,"Apple, Inc.",One Apple Park Way,None,None,Cupertino,California,95014-2083,United States,USA,...,Computers/Consumer Electronics,L,AAPL,A,None,http://www.apple.com,2046251,060704780,037833100,US0378331005


In [69]:
avro_df[avro_df['URL'].str.contains('apple.com', case=False)]

,_CompanyCode,CompanyName,Address1,Address2,Address3,City,State,PostalCode,Country,Region,...,IndustryDescriptor,ListStatus,ExchangeTicker,Status,RegistrationID,URL,SEDOL,DUNS,CUSIP,ISIN
56730,ZVHWML,"Pineapple Financial, Inc.",111 Gordon Baker Road,Unit 200,None,North York,Ontario,M2H 3R1,Canada,CANA,...,Mortgage Banks/Real Estate Credit,L,,A,None,http://www.gopineapple.com,None,None,None,
84480,ASIIRE,Apple Sales International Limited,Hollyhill Industrial Estate Hollyhill,None,None,Cork,Co Cork,None,Ireland,IRE,...,Computers/Consumer Electronics,UL,,A,157192,http://www.apple.com,None,238397210,None,
132854,APPLC,"Apple, Inc.",One Apple Park Way,None,None,Cupertino,California,95014-2083,United States,USA,...,Computers/Consumer Electronics,L,AAPL,A,None,http://www.apple.com,2046251,060704780,037833100,US0378331005
